In [1]:
# load model pretrained on MNLI
from transformers import BartForSequenceClassification, BartTokenizer
tokenizer = BartTokenizer.from_pretrained('facebook/bart-large-mnli')
model = BartForSequenceClassification.from_pretrained('facebook/bart-large-mnli')

'''
from transformers import AutoModelForSequenceClassification, AutoTokenizer
nli_model = AutoModelForSequenceClassification.from_pretrained('facebook/bart-large-mnli')
tokenizer = AutoTokenizer.from_pretrained('facebook/bart-large-mnli')
'''

c:\Users\Audric Ho\Downloads\LLM related stuff\open_source\zero_shot\bart_large_mnli\bart_large_mnli-venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\Audric Ho\Downloads\LLM related stuff\open_source\zero_shot\bart_large_mnli\bart_large_mnli-venv\Lib\site-packages\huggingface_hub\file_download.py:157: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Audric Ho\.cache\huggingface\hub\models--facebook--bart-large-mnli. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limi

"\nfrom transformers import AutoModelForSequenceClassification, AutoTokenizer\nnli_model = AutoModelForSequenceClassification.from_pretrained('facebook/bart-large-mnli')\ntokenizer = AutoTokenizer.from_pretrained('facebook/bart-large-mnli')\n"

In [ ]:
# pose sequence as a NLI premise and label as a hypothesis
'''
from transformers import AutoModelForSequenceClassification, AutoTokenizer
nli_model = AutoModelForSequenceClassification.from_pretrained('facebook/bart-large-mnli')
tokenizer = AutoTokenizer.from_pretrained('facebook/bart-large-mnli')
'''

premise = sequence
hypothesis = f'This example is {label}.'

# run through model pre-trained on MNLI
x = tokenizer.encode(premise, hypothesis, return_tensors='pt',
                     truncation_strategy='only_first')
logits = nli_model(x.to(device))[0]

# we throw away "neutral" (dim 1) and take the probability of
# "entailment" (2) as the probability of the label being true 
entail_contradiction_logits = logits[:,[0,2]]
probs = entail_contradiction_logits.softmax(dim=1)
prob_label_is_true = probs[:,1]


In [14]:
# load model pretrained on MNLI
'''
from transformers import BartForSequenceClassification, BartTokenizer
tokenizer = BartTokenizer.from_pretrained('facebook/bart-large-mnli')
model = BartForSequenceClassification.from_pretrained('facebook/bart-large-mnli')
'''

# pose sequence as a NLI premise and label (politics) as a hypothesis
premise = 'Who are you voting for in 2020?'
hypothesis = 'This text is about food.'
#hypothesis = 'This text is about politics.'

# run through model pre-trained on MNLI
input_ids = tokenizer.encode(premise, hypothesis, return_tensors='pt')
logits_all = model(input_ids)
print(f"logits_all: {logits_all}")
logits = logits_all[0]
print(f"logits: {logits}")

# we throw away "neutral" (dim 1) and take the probability of
# "entailment" (2) as the probability of the label being true 
entail_contradiction_logits = logits[:,[0,2]]
print(f"entail_contradiction_logits: {entail_contradiction_logits}")
probs = entail_contradiction_logits.softmax(dim=1)
print(f"probs: {probs}")
true_prob = probs[:,1].item() * 100
print(f'Probability that the label is true: {true_prob:0.2f}%')

logits_all: Seq2SeqSequenceClassifierOutput(loss=None, logits=tensor([[ 4.9938, -1.6333, -3.5080]], grad_fn=<AddmmBackward0>), past_key_values=((tensor([[[[ 3.5664e-01,  3.4130e-01,  2.2136e+00,  ...,  1.1739e+00,
            3.2585e+00, -1.4553e+00],
          [ 3.8734e-01,  4.8377e-01,  2.1573e+00,  ...,  1.6858e+00,
            3.9412e+00, -1.9962e+00],
          [-9.1104e-01, -1.3453e+00, -3.0937e+00,  ..., -3.0373e+00,
           -5.3019e+00, -2.8351e+00],
          ...,
          [-1.3513e+00, -1.8461e+00, -1.4017e+00,  ..., -5.2163e+00,
           -2.9571e+00, -3.1682e-01],
          [-3.1672e+00,  1.0403e+00,  1.3587e-01,  ..., -1.4038e+00,
           -3.4251e+00, -1.3457e+00],
          [-5.3134e+00,  2.7061e+00,  3.1567e-01,  ..., -1.4723e+00,
           -6.7663e+00, -2.4095e+00]],

         [[-2.2513e+00, -2.5750e+00, -1.9645e-01,  ..., -6.4791e-01,
            3.8783e-02,  7.9893e-01],
          [-5.3976e-01, -1.9856e+00, -1.9493e-01,  ..., -1.2245e+00,
            3.7229e-

In [9]:
from transformers import pipeline
classifier = pipeline("zero-shot-classification",
                      model="facebook/bart-large-mnli")


Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


In [10]:
sequence_to_classify = "one day I will see the world"
candidate_labels = ['travel', 'cooking', 'dancing']
classifier(sequence_to_classify, candidate_labels)
#{'labels': ['travel', 'dancing', 'cooking'],
# 'scores': [0.9938651323318481, 0.0032737774308770895, 0.002861034357920289],
# 'sequence': 'one day I will see the world'}


{'sequence': 'one day I will see the world',
 'labels': ['travel', 'dancing', 'cooking'],
 'scores': [0.9938650727272034, 0.0032738083973526955, 0.002861041808500886]}

In [11]:
candidate_labels = ['travel', 'cooking', 'dancing', 'exploration']
classifier(sequence_to_classify, candidate_labels, multi_label=True)
#{'labels': ['travel', 'exploration', 'dancing', 'cooking'],
# 'scores': [0.9945111274719238,
#  0.9383890628814697,
#  0.0057061901316046715,
#  0.0018193122232332826],
# 'sequence': 'one day I will see the world'}


{'sequence': 'one day I will see the world',
 'labels': ['travel', 'exploration', 'dancing', 'cooking'],
 'scores': [0.994511067867279,
  0.9383885264396667,
  0.0057061901316046715,
  0.0018192894058302045]}

In [16]:
def classify_sentence(candidate_labels, sequence_to_classify, multi_label = True):
    result_dict = {}
    classifier_results = classifier(sequence_to_classify, candidate_labels, multi_label=multi_label)
    if type(classifier_results) != list:
        classifier_results = [classifier_results]
    for classifier_result in classifier_results:
        result_dict[classifier_result["sequence"]] = {label:label_prob for label,label_prob in zip(classifier_result["labels"], classifier_result["scores"])}
    return result_dict

In [17]:
classify_sentence(candidate_labels, sequence_to_classify)

{'one day I will see the world': {'travel': 0.994511067867279,
  'exploration': 0.9383885264396667,
  'dancing': 0.0057061901316046715,
  'cooking': 0.0018192894058302045}}

In [15]:
classifier(["one day i will see the world", "i will explore sweden next semester"], candidate_labels, multi_label=True)

[{'sequence': 'one day i will see the world',
  'labels': ['travel', 'exploration', 'dancing', 'cooking'],
  'scores': [0.994157612323761,
   0.92877596616745,
   0.005361784249544144,
   0.0016605753917247057]},
 {'sequence': 'i will explore sweden next semester',
  'labels': ['travel', 'exploration', 'dancing', 'cooking'],
  'scores': [0.9911717772483826,
   0.9684410691261292,
   0.0032393524888902903,
   0.00020078456145711243]}]

In [19]:
classify_sentence(candidate_labels, ["one day i will see the world", "i will explore sweden next semester"], multi_label=False)

{'one day i will see the world': {'travel': 0.8104696869850159,
  'exploration': 0.1847233921289444,
  'dancing': 0.0025745946913957596,
  'cooking': 0.0022323287557810545},
 'i will explore sweden next semester': {'travel': 0.5285199284553528,
  'exploration': 0.46481984853744507,
  'dancing': 0.004193010274320841,
  'cooking': 0.002467212500050664}}